In [3]:
#!/usr/bin/env python3

"""
.. module:: rt_assignment2
   :platform: Unix
   :synopsis: Python module for the node_a
.. moduleauthor:: Mohsen Kashefi

This code implements the node_a.

Subscriber:
    - /node_a/pose: Subscribes to the pose.
"""

import rospy
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
from assignment_2_2023.msg import Vel
from assignment_2_2023.msg import PlanningAction, PlanningGoal
from std_srvs.srv import SetBool
from actionlib_msgs.msg import GoalStatus
from ipywidgets import widgets, interact
from IPython.display import display
import threading

def publish_position_and_velocity(msg, pos_vel_publisher):
    """
    Publishes the current position and velocity information.

    :param msg: Odometry message containing position and velocity information.
    :type msg: nav_msgs.msg.Odometry
    :param pos_vel_publisher: Publisher for position and velocity information.
    :type pos_vel_publisher: rospy.Publisher
    """
    current_pos = msg.pose.pose.position
    current_vel_linear = msg.twist.twist.linear
    current_vel_angular = msg.twist.twist.angular

    pos_and_vel = Vel()
    pos_and_vel.pos_x = current_pos.x
    pos_and_vel.pos_y = current_pos.y
    pos_and_vel.vel_x = current_vel_linear.x
    pos_and_vel.vel_z = current_vel_angular.z

    pos_vel_publisher.publish(pos_and_vel)

def get_new_goal():
    """
    Retrieves a new goal from user input.

    :return: New goal and its coordinates
    :rtype: tuple(assignment_2_2023.msg.PlanningGoal, tuple(float, float))
    """
    input_x = x_input.value
    input_y = y_input.value

    goal = PlanningGoal()
    goal.target_pose.pose.position.x = input_x
    goal.target_pose.pose.position.y = input_y

    return goal, (input_x, input_y)

def handle_goal_commands(b):
    """
    Handles goal commands from the user.
    """
    command = command_dropdown.value

    if command == 'Define New Goal (d)':
        new_goal, new_goal_coords = get_new_goal()
        if new_goal:
            rospy.set_param('/des_pos_x', new_goal_coords[0])
            rospy.set_param('/des_pos_y', new_goal_coords[1])
            print(new_goal)
            action_client.send_goal(new_goal)
    elif command == 'Cancel Goal (q)':
        action_client.cancel_goal()

# Set up ROS environment variables
# %env ROS_MASTER_URI=http://localhost:11311
# %env ROS_IP=your_computer_ip_address
# %env ROS_PACKAGE_PATH=/path/to/your/catkin_workspace:$ROS_PACKAGE_PATH

# Initialize ROS node
rospy.init_node('set_target_client', anonymous=True)

# ROS publisher
pos_vel_publisher = rospy.Publisher("/pos_vel", Vel, queue_size=1)
action_client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
action_client.wait_for_server()

# Setup subscriber
rospy.Subscriber("/odom", Odometry, lambda msg: publish_position_and_velocity(msg, pos_vel_publisher))

# IPython widgets for user interface
command_dropdown = widgets.Dropdown(
    options=['Define New Goal (d)', 'Cancel Goal (q)'],
    description='Command:'
)

x_input = widgets.FloatText(
    description='X:'
)

y_input = widgets.FloatText(
    description='Y:'
)

submit_button = widgets.Button(
    description='Submit'
)

# Define callback function for submit button click
submit_button.on_click(handle_goal_commands)

# Display UI
display(command_dropdown, x_input, y_input, submit_button)

# Spin ROS
rospy.spin()


Dropdown(description='Command:', options=('Define New Goal (d)', 'Cancel Goal (q)'), value='Define New Goal (d…

FloatText(value=0.0, description='X:')

FloatText(value=0.0, description='Y:')

Button(description='Submit', style=ButtonStyle())